# Part 5. 데이터 사전 처리

머신러닝 등 데이터 분석의 정확도는 분석 데이터의 품질에 의해 좌우된다.  
데이터의 품질을 높이기 위해서는 누락 데이터, 중복 데이터 등 오류를 수정하고 분석 목적에 맞게 변형하는 과정이 필요하다.  
수집한 데이터를 분석에 적합하도록 사전 처리하는 방법을 살펴보자.  

## 1. 누락 데이터 처리

일반적으로 유효한 데이터 값이 존재하지 않는 누락 데이터를 NaN(Not a Number)으로 표시한다.  

누락 데이터가 많아지면 데이터의 품질이 떨어지고, 머신러닝 분석 알고리즘을 왜곡하는 현상이 발생하기 때문에, 분석 하기 전 반드시 누락 데이터를 제거하거나 다른 적절한 값으로 대체하는 과정이 필요하다.

* 누락 데이터 확인

In [1]:
import seaborn as sns

df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
df.info()
# 'deck' 열에 203 개의 유효한 데이터가 존재, 누락 데이터는 688(891-203)개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [3]:
nan_deck = df['deck'].value_counts(dropna=False) # dropna=False 옵션을 사용하지 않으면 유효한 데이터의 개수만을 구한다.
nan_deck

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64

누락 데이터를 찾는 직접적인 방법  
* **isnull()** : 누락 데이터면 True 반환, 유효한 데이터가 존재하면 False 반환  
* **notnull()** : 유효한 데이터가 존재하면 True 반환, 누락 데이터면 False 반환

In [4]:
print(df.head().isnull())
print('\n')
print(df.head().notnull())

   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  


   survived  pclass   sex   age  sibsp  parch  fare  embarked  class   who  \
0      True    True  True  True   True   True  True      True   True  True   
1      True    True  True  True   True   True  Tr

In [5]:
# 누락 데이터의 개수 구하기 

print(df.head().isnull().sum())

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64


* 누락 데이터 제거  

dropna() 메소드 사용

열을 삭제하면 분석 대상이 갖는 특성(변수)를 제거한다.  
행을 삭제하면 분석 대상의 관측값(레코드)을 제거하게 된다.

In [6]:
import seaborn as sns

df = sns.load_dataset('titanic')

missing_df = df.isnull()
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts() # 각 열의 NaN 개수 파악
    
    try: 
        print(col, ":", missing_count[True])
    except:
        print(col, ":", 0)

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


위 결과에서 'deck'열의 누락 데이터가 차지하는 비율이 매우 높기 때문에   
'deck'열의 누락 데이터를 삭제해 분석에서 제외하는 것이 의미 있다.

In [7]:
# NaN 값이 500개 이상인 열을 모두 삭제 - deck 열 삭제
df_thresh = df.dropna(axis=1, thresh=500)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


In [8]:
# age 열에 나이 데이터가 없는 모든 행 삭제
df_age = df.dropna(subset=['age'], how='any', axis=0)
print(len(df_age))

714


how='any' 옵션: NaN값이 하나라도 존재하면 삭제한다.(default)  
how='all' 옵션: 모든 데이터가 NaN값일 경우에만 삭제

* 누락 데이터 치환  

fillna() 메소드 사용  
원본 객체를 변경하려면 inplace=True 옵션을 추가해야 한다.

In [9]:
df['age'].head(10)

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64

In [10]:
# NaN값을 다른 나이 데이터의 평균으로 변경
mean_age = df['age'].mean() # 평균 계산
df['age'].fillna(mean_age, inplace=True)

print(df['age'].head(10))

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


In [11]:
df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object

In [12]:
# NaN값을 가장 많이 출현한 값으로 치환하기 (idxmax() 메소드 사용)
most_freq = df['embark_town'].value_counts(dropna=True).idxmax() #최빈값 계산
print(most_freq)

df['embark_town'].fillna(most_freq, inplace=True)

df['embark_town'][825:830]

Southampton


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object

> **누락 데이터가 NaN으로 표시되지 않는 경우**
>
> 숫자 0이나 문자 '-', '?' 같은 값으로 입력되기도 한다.  
> 판다스에서 누락 데이터를 다루려면 replace() 메소드를 활용해 NumPy에서 지원하는 np.nan으로 변경해주는 것이 좋다.
>
> * 사용법 예) `df.replace('?', np.nan, inplace=True)`

데이터셋 특성상 서로 이웃하고 있는 데이터끼리 유사성을 가질 가능성이 높다.  

fillna() 메소드에  
method='ffill'('pad') 추가     : NaN이 있는 행의 직전 행에 있는 값으로 치환  
method='bfill'('backfill') 추가 : NaN이 있는 행의 바로 다음 행에 있는 값으로 치환

In [13]:
import seaborn as sns

df = sns.load_dataset('titanic')

df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object

In [14]:
df['embark_town'].fillna(method='ffill', inplace=True)

df['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object

## 2. 중복 데이터 처리

* 중복 데이터 확인  

duplicated() 메소드 사용  
-> 이전 행들과 비교해 중복되는 행이면 True, 처음 나오는 행이면 False

In [15]:
import pandas as pd

df = pd.DataFrame({'c1':['a','a','b','a','b'],
                  'c2':[1,1,1,2,2],
                  'c3':[1,1,2,2,2]})
print(df)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [16]:
df_dup = df.duplicated()
print(df_dup)

0    False
1     True
2    False
3    False
4    False
dtype: bool


In [17]:
col_dup = df['c2'].duplicated()
print(col_dup)

0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


* 중복 데이터 제거  

drop_duplicates() 메소드 사용  
원본 객체를 변경하려면 inplace=True 옵션 추가

In [18]:
import pandas as pd

df = pd.DataFrame({'c1':['a','a','b','a','b'],
                  'c2':[1,1,1,2,2],
                  'c3':[1,1,2,2,2]})
print(df)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [19]:
df2 = df.drop_duplicates()
print(df2)

  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [20]:
# c2, c3열을 기준으로 중복 행 제거
df3 = df.drop_duplicates(subset=['c2','c3'])
print(df3)

  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2


## 3. 데이터 표준화

서로 다른 단위들이 섞여 있거나 같은 대상을 다른 형식으로 표현하는 등 동일한 대상을 표현하는 방법에 차이가 있으면, 분석의 정확도는 현저히 낮아진다.  
따라서 데이터 포맷을 일관성 있게 표준화하는 작업이 필요하다.  

### 3-1. 단위 환산  

같은 데이터셋 안에서 측정 단위를 동일하게 맞춰야 일관성 측면에서 문제가 발생하지 않는다.  
영미권에서 주로 사용하는 마일, 야드, 온스 등의 도량형 단위를 한국에서 사용하는 미터, 평, 그램 등으로 반환하는 것이 좋다.

In [21]:
import pandas as pd

df = pd.read_csv('auto-mpg.csv', header=None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
             'acceleration', 'model year', 'origin', 'name']
print(df.head(3))

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  


In [22]:
# mpg(mile per gallson)를 kpl(kilometer per liter)로 변환(1mpg = 0.425km/L)
mpg_to_kpl = 1.60934/3.78541

df['kpl'] = df['mpg'] * mpg_to_kpl
print(df.head(3))
print('\n')
df['kpl'] = df['kpl'].round(2) # 소수점 아래 둘째자리에서 반올림
print(df.head(3))

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name       kpl  
0       1  chevrolet chevelle malibu  7.652571  
1       1          buick skylark 320  6.377143  
2       1         plymouth satellite  7.652571  


    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name   kpl  
0       1  chevrolet chevelle malibu  7.65  
1       1          buick skylark 320  6.38  
2       1        

### 3-2. 자료형 변환

먼저 dtypes 속성 혹은 info() 메소드로 각 열의 자료형을 확인 후   
astype() 메소드를 사용해 변환

In [23]:
print(df.dtypes)

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight          float64
acceleration    float64
model year        int64
origin            int64
name             object
kpl             float64
dtype: object


'horsepower' 열은 문자열을 뜻하는 object 자료형이다.  
그러나 엔진 출력의 크기를 나타내는 데이터인 만큼 숫자형으로 변환해 주는 것이 적잘하다.

In [24]:
print(df['horsepower'].unique())
# 문자 '?'가 섞여 있어 문자열로 인식된 것으로 보인다.

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '?' '100.0' '105.0' '175.0' '153.0'
 '180.0' '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00'
 '80.00' '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0'
 '167.0' '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00'
 '83.00' '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00'
 '98.00' '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0'
 '68.00' '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0'
 '125.0' '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00'
 '64.00' '74.00' '116.0' '82.00']


In [25]:
# 누락 데이터('?') 삭제
import numpy as np

df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)

# 문자열을 실수형으로 변환
df['horsepower'] = df['horsepower'].astype('float')
print(df['horsepower'].dtypes)

float64


'origin' 열은 정수형 데이터 1, 2, 3 을 갖고 있지만 실제로는 국가 이름인 'USA, EU, JPN' 을 뜻한다.  
replace() 메소드를 사용해 각 숫자 데이터를 국가이름으로 바꿔주면 문자열을 나타내는 object 자료형으로 자동 변경된다.

In [26]:
print(df['origin'].unique())

df['origin'].replace({1:'USA', 2:'EU', 3:'JPN'}, inplace=True)

print(df['origin'].unique())
print(df['origin'].dtypes)

[1 3 2]
['USA' 'JPN' 'EU']
object


In [27]:
# 문자열을 범주형으로 변환
df['origin'] = df['origin'].astype('category')
print(df['origin'].dtypes)

# 범주형을 다시 문자형으로 변환
df['origin'] = df['origin'].astype('str')
print(df['origin'].dtypes)

category
object


'model year' 열은 모델 출시년도가 숫자로 기록되어 있고 자료형은 정수형을 나타내는 int64이다.  
연도는 시간적인 순서의 의민ㄴ 있으나 숫자의 상대적인 크기는 별 의미가 없다.  
따라서 데이터는 숫자 형태를 갖더라도 자료형은 범주형으로 표현하는 것이 적절하다.

In [28]:
print(df['model year'].sample(3))
print('\n')
df['model year'] = df['model year'].astype('category')
print(df['model year'].sample(3))

0      70
128    74
135    74
Name: model year, dtype: int64


26     70
280    79
226    77
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]


## 4. 범주형(카테고리) 데이터 처리

### 4-1. 구간 분할

구간 분할: 연속 변수를 일정한 구간으로 나누고, 각 구간을 범주형 이산 변수로 변환하는 과정  
cut() 함수를 이용한다.  

경계값은 histogram() 함수의 bins 옵션을 활용해 구한다.

In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv('auto-mpg.csv', header=None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
             'acceleration', 'model year', 'origin', 'name']

df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True) # 누락 데이터 행 삭제
df['horsepower'] = df['horsepower'].astype('float')

# 3개의 bin으로 나누는 경계값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


In [30]:
bin_names = ['저출력', '보통출력', '고출력']

# 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],
                     bins=bin_dividers,    # 경계값 리스트
                     labels=bin_names,     # bin 이름
                     include_lowest=True)  # 각 구간의 낮은 경계값(첫 경계값) 포함

print(df[['horsepower', 'hp_bin']].head(15))

    horsepower hp_bin
0        130.0   보통출력
1        165.0   보통출력
2        150.0   보통출력
3        150.0   보통출력
4        140.0   보통출력
5        198.0    고출력
6        220.0    고출력
7        215.0    고출력
8        225.0    고출력
9        190.0    고출력
10       170.0    고출력
11       160.0   보통출력
12       150.0   보통출력
13       225.0    고출력
14        95.0    저출력


### 4-2. 더미 변수

get_dummies() 함수 사용

카테고리를 나타내는 범주형 데이터를 회귀분석 등 머신러닝 알고리즘에 바로 사용할 수 없는 경우가 있는데, 컴퓨터가 인식 가능한 입력값으로 변환해야 한다.  
이럴 때 숫자 0 또는 1로 표현되는 더미 변수를 사용한다.  
여기서 0과 1은 어떤 특성이 있는지 없는지 여부만을 표시한다.

==> one-hot-coding 이라고도 부른다.

In [31]:
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))

    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


In [32]:
# sklearn 라이브러리 이용해 원핫 인코딩 처리하기 ==> 희소행렬로 정리
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder() # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder() # one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 형태 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))

# 희소행렬로 변환 -> (행, 열) 값
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


## 5. 정규화

숫자 데이터의 상대적인 크기 차이를 제거할 필요가 있다.  
정규화: 각 열에 속하는 데이터 값을 동일한 크기 기준으로 나눈 비율로 나타내는 것  
이 후 데이터의 범위는 0\~1 또는 -1\~1이 된다.

1. 각 열의 데이터를 해당 열의 최대값으로 나누기

In [33]:
import pandas as pd
import numpy as np

df = pd.read_csv('auto-mpg.csv', header=None)

df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
             'acceleration', 'model year', 'origin', 'name']

df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True) # 누락 데이터 행 삭제
df['horsepower'] = df['horsepower'].astype('float')

print(df.horsepower.describe()) # 최대값 확인
print('\n')

# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
df.horsepower = df.horsepower/abs(df.horsepower.max())

print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64


count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


2. 각 열의 데이터 중에서 최대값과 최소값을 뺀 값으로 나누기

In [34]:
# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x/min_max

print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())

0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64


count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64


## 6. 시계열 데이터

시계열 데이터를 데이터프레임의 행 인덱스로 사용하면, 시간으로 기록된 데이터를 분석하는 것이 매우 편리하다.  
판다스의 시간 표시 방식  
**Timestamp**: 특정한 시점 기록  
**Period**: 두 시점 사이의 일정한 기간 나타냄

## 6-1. 다른 자료형을 시계열 객체로 변환

* 문자열을 Timestamp로 변환  

판다스 to_datetime() 함수를 사용하면 문자열 등 다른 자료형을 판다스 Timestamp를 나타내는 datetime64 자료형으로 변환 가능하다.

In [35]:
import pandas as pd

df = pd.read_csv('stock-data.csv')

print(df.head())
print('\n')
print(df.info())

         Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None


In [36]:
df['new_Date'] = pd.to_datetime(df['Date'])

print(df.head())
print('\n')
print(df.info())
print('\n')
print(type(df['new_Date'][0]))

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB
None


<class 'pandas._libs.tslibs.timestamps.T

In [37]:
# 시계열 값으로 변환된 열을 새로운 행 인덱스로 지정, 기존 날짜 열 삭제
# ==> 시간 순서에 맞춰 인덱싱 또는 슬라이싱 하기가 편리하다.
df.set_index('new_Date', inplace=True)
df.drop('Date', axis=1, inplace=True)

print(df.head())
print('\n')
print(df.info())

            Close  Start   High    Low  Volume
new_Date                                      
2018-07-02  10100  10850  10900  10000  137977
2018-06-29  10700  10550  10900   9990  170253
2018-06-28  10400  10900  10950  10150  155769
2018-06-27  10900  10800  11050  10500  133548
2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Close   20 non-null     int64
 1   Start   20 non-null     int64
 2   High    20 non-null     int64
 3   Low     20 non-null     int64
 4   Volume  20 non-null     int64
dtypes: int64(5)
memory usage: 960.0 bytes
None


* Timestamp를 Period로 변환  

판다스 to_period() 함수를 이용하면 일정한 기간을 나타내는 Period 객체로 Timestamp 객체를 변환할 수 있다.  
freq 옵션에 기준이 되는 기간을 설정한다.  
  
**freq 옵션의 종류**

옵션  | 설명
------|------
    D | day(1일)
    W | week(1주)
    M | month end(월말)
    MS| month begin(월초)
    Q | quarter end(분기말)
    QS| quarter begin(분기초)
    A | year end(연말)
    AS| year begin(연초)
    B | business day(휴일 제외)
    H | hour(1시간)
    T | minute(1분)
    S | second(1초)
    L | milisecond
    U | microsecond
    N | nanosecond
   ...| ...

In [38]:
import pandas as pd

dates = ['2019-01-01', '2020-03-01', '2021-06-01']

# 문자열의 배열을 판다스 Timestamp로 변환
ts_dates = pd.to_datetime(dates)
print(ts_dates)
print('\n')

# Timestamp를 Period로 변환
pr_day = ts_dates.to_period(freq='D')
print(pr_day)
pr_month = ts_dates.to_period(freq='M')
print(pr_month)
pr_year = ts_dates.to_period(freq='A')
print(pr_year)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)


PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')


### 6-2. 시계열 데이터 만들기

* Timestamp 배열   

판다스 date_range() 함수 사용


In [39]:
import pandas as pd

ts_ms = pd.date_range(start='2019-01-01',
                     end=None,         # 날짜 범위 끝
                     periods=6,        # 생성할 Timestamp 개수
                     freq='MS',        # 시간 간격
                     tz='Asia/Seoul')  # 시간대(timezone)

print(ts_ms)

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [40]:
ts_me = pd.date_range('2019-01-01', periods=6,      
                     freq='M',      
                     tz='Asia/Seoul') 
print(ts_me)

DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


In [41]:
# 분기 (3개월) 간격
ts_3m = pd.date_range('2019-01-01', periods=6,      
                     freq='3M',      
                     tz='Asia/Seoul') 
print(ts_3m)

DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')


* Period 배열  

판다스 period_range() 함수는 여러 개의 기간이 들어 있는 시계열 데이터를 만든다.

In [42]:
import pandas as pd

pr_m = pd.date_range(start='2019-01-01', 
                     end=None,
                     periods=3,      # 생성할 Period 개수
                     freq='M')       # 기간의 길이(M: 월)
print(pr_m)

DatetimeIndex(['2019-01-31', '2019-02-28', '2019-03-31'], dtype='datetime64[ns]', freq='M')


In [43]:
pr_h = pd.date_range(start='2019-01-01', 
                     end=None,
                     periods=3,     
                     freq='H')    
print(pr_h)

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 01:00:00',
               '2019-01-01 02:00:00'],
              dtype='datetime64[ns]', freq='H')


In [44]:
pr_2h = pd.date_range(start='2019-01-01', 
                      end=None,
                      periods=3,     
                      freq='2H')    
print(pr_2h)

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 02:00:00',
               '2019-01-01 04:00:00'],
              dtype='datetime64[ns]', freq='2H')


### 6-3. 시계열 데이터 활용

* 날짜 데이터 분리

1. dt 속성 이용

In [45]:
import pandas as pd

df = pd.read_csv('stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])

print(df.head())
print('\n')

df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26


2. to_period() 메소드 적용

In [46]:
# Timestamp를 Period로 변환하여 연-월-일 표기 변경

df['Date_yr'] = df['new_Date'].dt.to_period(freq='A')
df['Date_m'] = df['new_Date'].dt.to_period(freq='M')
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  \
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7   
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   

   Day Date_yr   Date_m  
0    2    2018  2018-07  
1   29    2018  2018-06  
2   28    2018  2018-06  
3   27    2018  2018-06  
4   26    2018  2018-06  


In [47]:
df.set_index('Date_m', inplace=True)
print(df.head())

               Date  Close  Start   High    Low  Volume   new_Date  Year  \
Date_m                                                                     
2018-07  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018   
2018-06  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018   
2018-06  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018   
2018-06  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018   
2018-06  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018   

         Month  Day Date_yr  
Date_m                       
2018-07      7    2    2018  
2018-06      6   29    2018  
2018-06      6   28    2018  
2018-06      6   27    2018  
2018-06      6   26    2018  


* 날짜 인덱스 활용  

시계열 데이터에 대한 인덱싱과 슬라이싱 편리하다.  
내가 필요로 하는 레벨을 선택적으로 인덱싱할 수 있다. (연도, 연-월, 연-월-일...)

In [52]:
import pandas as pd

df = pd.read_csv('stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])
df.set_index('new_Date', inplace=True)

print(df.head())
print('\n')
print(df.index)

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


DatetimeIndex(['2018-07-02', '2018-06-29', '2018-06-28', '2018-06-27',
               '2018-06-26', '2018-06-25', '2018-06-22', '2018-06-21',
               '2018-06-20', '2018-06-19', '2018-06-18', '2018-06-15',
               '2018-06-14', '2018-06-12', '2018-06-11', '2018-06-08',
               '2018-06-07', '2018-06-05', '2018-06-04', '2018-06-01'],
              dtype='datetime64[ns]', name='new_Date', freq=None)


In [54]:
df_y = df.loc['2018']
print(df_y.head())
print('\n')
df_ym = df.loc['2018-07']
print(df_ym)
print('\n')
df_ym_cols = df.loc['2018-07', 'Start':'High'] # 열 범위 슬라이싱
print(df_ym_cols)
print('\n')
df_ymd = df.loc['2018-07-02']
print(df_ymd)
print('\n')
df_ymd_range = df.loc['2018-06-20':'2018-06-25']
print(df_ymd_range)

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977


            Start   High
new_Date                
2018-07-02  10850  10900


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-06-25  2018-06-25  11150  1

In [50]:
# 시간 간격 계산
today = pd.to_datetime('2018-12-25')
df['time_delta'] = today - df.index
df.set_index('time_delta', inplace=True)
df_180 = df['180 days':'189 days'] # 최근 180일 ~ 189일 사이의 값들 선택
print(df_180)

                  Date  Close  Start   High    Low  Volume
time_delta                                                
180 days    2018-06-28  10400  10900  10950  10150  155769
181 days    2018-06-27  10900  10800  11050  10500  133548
182 days    2018-06-26  10800  10900  11000  10700   63039
183 days    2018-06-25  11150  11400  11450  11000   55519
186 days    2018-06-22  11300  11250  11450  10750  134805
187 days    2018-06-21  11200  11350  11750  11200  133002
188 days    2018-06-20  11550  11200  11600  10900  308596
189 days    2018-06-19  11300  11850  11950  11300  180656
